In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
!pip install pydriller

In [ ]:
#Get large corpus for training the wrod2vec model
from pydriller import Repository

repos = ["https://github.com/numpy/numpy", "https://github.com/django/django", "https://github.com/scikit-learn/scikit-learn", "https://github.com/tensorflow/tensorflow", "https://github.com/keras-team/keras","https://github.com/ansible/ansible", "https://github.com/TheAlgorithms/Python", "https://github.com/pallets/flask", "https://github.com/ytdl-org/youtube-dl", "https://github.com/pandas-dev/pandas", "https://github.com/scrapy/scrapy", "https://github.com/kennethreitz/requests", "https://github.com/home-assistant/home-assistant", "https://github.com/ageitgey/face_recognition","https://github.com/emesik/mamona","https://github.com/progrium/notify-io","https://github.com/phoenix2/phoenix","https://github.com/odoo/odoo","https://github.com/ageitgey/face_recognition","https://github.com/psf/requests","https://github.com/deepfakes/faceswap","https://github.com/XX-net/XX-Net","https://github.com/tornadoweb/tornado","https://github.com/saltstack/salt","https://github.com/matplotlib/matplotlib","https://github.com/celery/celery","https://github.com/binux/pyspider","https://github.com/miguelgrinberg/flasky","https://github.com/sqlmapproject/sqlmap","https://github.com/zulip/zulip","https://github.com/scipy/scipy","https://github.com/bokeh/bokeh","https://github.com/docker/compose","https://github.com/getsentry/sentry","https://github.com/timgrossmann/InstaPy","https://github.com/divio/django-cms","https://github.com/boto/boto"]

pythontraining = ""

for r in repos:
  print(r)
  files = []
  for commit in Repository(r).traverse_commits():
      for m in commit.modified_files:
        filename = m.new_path
        
        if filename is not None:
          if ".py" in filename:
            if not filename in files:
              code = m.source_code
              if code is not None:
                pythontraining = pythontraining + "\n\n" + code
                files.append(filename)
        
          
  with open('/drive/MyDrive/pythontraining.txt', 'w') as outfile:
    outfile.write(pythontraining)

https://github.com/numpy/numpy


KeyboardInterrupt: ignored

In [ ]:
!pip install cStringIO

ERROR: Could not find a version that satisfies the requirement cStringIO (from versions: none)
ERROR: No matching distribution found for cStringIO


In [ ]:
#clean the downloaded corpus

#import sys
import subprocess
import time

#remove some broken code in the corpus

f=open("/drive/MyDrive/pythontraining.txt", "r")
contents =f.read()
contents = contents.replace('\t', '    ')

if 'PositiveSmallIntegerField(\n                choices' in contents:
    pos = contents.find('PositiveSmallIntegerField(\n                choices')
    contents = contents[:pos-198] + contents[pos+178:]

if "            raise ImportError,self.__dict__.get('_ppimport_exc_info')[1]" in contents:
  pos = contents.find("            raise ImportError,self.__dict__.get('_ppimport_exc_info')[1]")
  length = len("            raise ImportError,self.__dict__.get('_ppimport_exc_info')[1]")
  contents = contents[:pos] + contents[pos+length+1:]

if "[k]*step+start)" in contents:
  pos = contents.find("[k]*step+start)")
  contents = contents[:pos+17] + contents[pos+21:]
badstring = ["silly_field", "('id', models.AutoField(primary_key=True))"]

while "check_framework.Model2." in contents:
  pos = contents.find("check_framework.Model2.")
  area = contents[pos-300:pos+300]
  start = area.find("class")
  end = area.find("def")  
  contents = contents[:pos-300+start] + contents[pos-300+end:]

fromhere = 0
while "DEFAULT_KMS_KEY_NAME" in contents[fromhere:] and "ENCRYPTION_CONFIG" in contents[fromhere:fromhere+2000]:
  pos = fromhere + contents[fromhere:].find("DEFAULT_KMS_KEY_NAME")
  area = contents[pos-1000:pos+1000]
  start = area[:1000].find("class")
  if (start == -1):
    start = area[:1000].find("from")
  if (start == -1):
    start = area[:1000].find("import")
  if (start == -1):
    start = area[:1000].find("def")
    
  end = area[1000:].find("def")
  if (end == -1):
    end = area[1000:].find("from")
  if (end == -1):
    end = area[1000:].find("import")
  
  print("Found it at  " + str(pos))
#    print(len(contents))
  if (start > 0 and end > 0):
    contents = contents[:pos-1000+start] + contents[pos-1000+end:]
    fromhere = pos-1000+start+end+1
    print("countinue at " + str(fromhere))
    print(start)
    print(end)
  else:
    fromhere = pos + 1000
  
 
fromhere = 0
while "somepassword" in contents[fromhere:]:
  pos = fromhere + contents[fromhere:].find("somepassword")
  area = contents[pos-1000:pos+1000]
  start = area.find("def")
  end = area[1000:].find("def")
  if (end == -1):
    end = area[1000:].find("from")
  if (end == -1):
    end = area[1000:].find("import")
  if start > 0 and end > 0:
    contents = contents[:pos-1000+start] + contents[pos+end:]
    fromhere = pos-1000+start

  else:
    fromhere = pos + 1
  
  
if "somepassword" in contents and "someuser" in contents and "somehost" in contents:
  pos = contents.find("somepassword")

for x in badstring:
  while(x in contents):    
    pos = contents.find(x)    
    area = contents[pos-500:pos+1000]            
    if("db.create_table" in area):
      contents = contents.replace("('id', models.AutoField(primary_key=True))","('id', models.AutoField(primary_key=False))",1)
      continue    
    start = area.find("class")    
    restarea = area[start:]    
    end = restarea.find("from") + start
    end2 = restarea.find("import") + start    
    if end2 < end:
      end = end2 
    if (end > start):
      contents = contents[:pos-500+start] + contents[pos-500+end:]
 
 
f = open("/drive/MyDrive/cleanpythontraining.py", "w")
f.write(contents)
f.close()    

In [ ]:
!pip install tokenizer

     |████████████████████████████████| 77 kB 2.8 MB/s 


In [ ]:
#tokenize the clean data:
!tokenize /content/drive/MyDrive/cleanpythontraining.txt /content/drive/MyDrive/tokenizepython.txt

Traceback (most recent call last):
  File "/usr/local/bin/tokenize", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/tokenizer/main.py", line 257, in main
    for t in tokenize(gen(args.infile), **options):
  File "/usr/local/lib/python3.7/dist-packages/tokenizer/tokenizer.py", line 2940, in <genexpr>
    return (t for t in token_stream if t.kind != TOK.X_END)
  File "/usr/local/lib/python3.7/dist-packages/tokenizer/tokenizer.py", line 2802, in parse_phrases_2
    next_token = next(token_stream)
  File "/usr/local/lib/python3.7/dist-packages/tokenizer/tokenizer.py", line 2639, in parse_date_and_time
    next_token = next(token_stream)
  File "/usr/local/lib/python3.7/dist-packages/tokenizer/tokenizer.py", line 2520, in parse_phrases_1
    next_token = next(token_stream)
  File "/usr/local/lib/python3.7/dist-packages/tokenizer/tokenizer.py", line 2400, in parse_sentences
    next_token = next(token_stream)
  File "/usr/local/lib/python3.7/dist-pac

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
import nltk
nltk.download('punkt')
from gensim.models import Word2Vec, KeyedVectors
import pickle
import io

text = []

with open ('/content/drive/MyDrive/tokenizepython.txt') as f:
  data = f.read().lower().replace('\n','')

print("length of training file: ",str(len(data)))
print("It contains " + str(data.count(" ")) + " individual code tokens.")

corpus = nltk.sent_tokenize(data)
text = [nltk.word_tokenize(sent) for sent in corpus]

with open('/content/drive/MyDrive/finalprocessed','wb') as f:
  pickle.dump(text,f)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
length of training file:  0
It contains 0 individual code tokens.


In [ ]:
for mincount in [10,30,50,100,300,500,5000]:
  for iterationen in [1,5,10,30,50,100]:
    for s in [5,10,15,30,50,75,100,200,300]:
      fname = "/content/drive/MyDrive/word2vec_"+str(mincount) + "-" + str(iterationen) +"-" + str(s)+ ".model"

      model = Word2Vec(text, size=s, min_count=mincount, iter=iterationen, workers = 4) 
      vocabulary = model.wv.vocab

      model.save(fname)

RuntimeError: ignored

In [ ]:
#Train word2vec

import re
import time
from tqdm.auto import tqdm
from collections import Counter

import nltk

from nltk import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim.models import Word2Vec

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#Calculate the time elapsed
def cal_elapsed_time(s):
    return print("Elapsed time:\t", round((time.time() - s),2))
s_time = time.time()
cal_elapsed_time(s=s_time)

Elapsed time:	 0.0


In [ ]:
#Model Training

from gensim.models import Word2Vec

s_time = time.time()
print("Model Training Started...")
model = Word2Vec(sentences=text, size=200, window=4, min_count=1, workers=4)
cal_elapsed_time(s_time)

Model Training Started...


RuntimeError: ignored

In [ ]:
print("Total number of unique words loaded in Model : ", len(model.wv.vocab))

Total number of unique words loaded in Model :  95634


In [ ]:
model.save("/drive/MyDrive/trainned_model.model")

In [ ]:
model = Word2Vec.load('/drive/MyDrive/trainned_model.model')

In [ ]:
model.wv.most_similar("efficiency", topn=10)

[('purposes', 0.7259385585784912),
 ('non-posix', 0.7037169933319092),
 ('sake', 0.6968435049057007),
 ('visualisation', 0.68260657787323),
 ('determining', 0.6761747598648071),
 ('shift-inverse', 0.6625900268554688),
 ('1.1.4', 0.6568349599838257),
 ('anisotropy', 0.6536865234375),
 ('lipschitz', 0.6529261469841003),
 ('searches', 0.6446515321731567)]

In [ ]:
model.wv.get_vector('import')

array([ 1.44902611e+00,  8.72539699e-01, -4.29428041e-01, -2.20043731e+00,
        7.54272714e-02,  1.02773106e+00,  9.84508157e-01,  1.92134053e-01,
        1.12890649e+00, -1.38815641e+00,  1.99967885e+00,  2.87475038e+00,
       -3.65889072e+00,  4.60960656e-01,  3.47248483e+00, -1.72620785e+00,
        5.08278668e-01, -3.73129106e+00, -2.56346405e-01, -9.20481682e-01,
        1.87428907e-01, -2.33639288e+00, -2.79611039e+00,  2.32214642e+00,
       -2.15983844e+00,  2.62277913e+00,  5.78224480e-01, -2.01162982e+00,
       -1.20323944e+00, -2.17173243e+00, -1.23576927e+00,  1.86194468e+00,
       -1.00383008e+00,  4.85082984e-01, -1.00634563e+00,  3.44605278e-03,
        2.01964545e+00, -2.73886299e+00,  3.28830671e+00, -1.18899775e+00,
       -2.85535002e+00, -1.03160553e-01,  2.85433578e+00,  1.47422767e+00,
       -3.02268815e+00,  1.77371240e+00, -4.74226803e-01,  2.04151440e+00,
       -1.03171885e+00, -1.56458914e+00,  9.98746812e-01, -1.42508054e+00,
       -1.90015006e+00, -

In [ ]:
#Scrape Github 
import os
import requests
import time
import sys
import json
from requests_oauthlib import OAuth1Session
from requests_oauthlib import OAuth1

def gettoken():
  with open('/drive/MyDrive/token.txt', 'r') as token:
        gittoken = token.readline().replace("\n","")
  return gittoken

#print(gettoken())


def getgitresponse(fixes,commits):
  #collect links from the github API response
  maximum = 9999  
  new = 0


  #craft request for Github
  params = (
      ('q', fixes),('per_page',100)
  )
  
  headers = {'Accept': 'application/vnd.github.cloak-preview', 'Authorization': 'token ' + gettoken()}
  url = "https://api.github.com/search/commits"
 
  for i in range(0,maximum):
      print(str(len(commits)) + " commits so far.")
      #print
      limit = 0
      while(limit == 0):
          #request search results
          response = requests.get(url, headers=headers,params=params)
          h = response.headers
          if 'X-RateLimit-Remaining' in h:
            limit = int(h['X-RateLimit-Remaining'])
            if limit == 0:
                # Limit of requests per time was reached, sleep to wait until we can request again
                print("Rate limit. Sleep.")
                time.sleep(35)
            #else:
            #  print(h)
      if 'Link' not in h:
        break;
      
      #go through all elements in Github's reply
      content = response.json()
      for j in range(0, len(content["items"])):
          #get relevant info
          repo = content["items"][j]["repository"]["html_url"]
          if repo not in commits:
              #new repository, new commit
              c = {}
              c["url"] = content["items"][j]["url"]
              c["html_url"] = content["items"][j]["html_url"]
              c["message"] = content["items"][j]["commit"]["message"]
              c["sha"] = content["items"][j]["sha"]
              c["keyword"] = fixes
              commits[repo] = {}
              commits[repo][content["items"][j]["sha"]] = c;
          else:
              if not content["items"][j]["sha"] in commits[repo]:
                #new commit for this already known repository
                new = new + 1
                c = {}
                c["url"] = content["items"][j]["url"]
                c["html_url"] = content["items"][j]["html_url"]
                c["sha"] = content["items"][j]["sha"]
                c["keyword"] = fixes
                commits[repo][content["items"][j]["sha"]] = c;
                
                
      #get the links to the next results
      link = h['Link']
      reflinks = analyzelinks(link)
      if "last" in reflinks:
          lastnumber = reflinks["last"].split("&page=")[1]
          maximum = int(lastnumber)-1
      if not "next" in reflinks:
          #done with all that could be collected
          break
      else:
          nextlink = reflinks["next"]
          
  #save the commits that were found
  with open('/drive/MyDrive/commits.json', 'w') as outfile:
      json.dump(commits, outfile)



def analyzelinks(link):
    #get references to the next page of results
    
    link = link + ","
    reflinks = {}
    while "," in link:
        pos = link.find(",")
        text = link[:pos]
        rest = link[pos+1:]
        try:
          if "\"next\"" in text:
              text = text.split("<")[1]
              text = text.split(">;")[0]
              reflinks["next"]=text
          if "\"prev\"" in text:
              text = text.split("<")[1]
              text = text.split(">;")[0]
              reflinks["prev"]=text
          if "\"first\"" in text:
              text = text.split("<")[1]
              text = text.split(">;")[0]
              reflinks["first"]=text
          if "\"last\"" in text:
              text = text.split("<")[1]
              text = text.split(">;")[0]
              reflinks["last"]=text
        except IndexError as e:
            print(e)
            print("\n")
            print(text)
            print("\n\n")
            sys.exit()
        link = rest
    return(reflinks)





#------------------------------------




commits = {}

#load previously scraped commits
with open('/drive/MyDrive/commits.json', 'r') as jsonfile:
    data = jsonfile.read()
    

vulnerabilities = ["buffer overflow","denial of service", "dos", "XXE","vuln","CVE","XSS","NVD","malicious","cross site","exploit","directory traversal","rce","remote code execution","XSRF","cross site request forgery","click jack","clickjack","session fixation","cross origin","infinite loop","brute force","buffer overflow","cache overflow","command injection","cross frame scripting","csv injection","eval injection","execution after redirect","format string","path disclosure","function injection","replay attack","session hijacking","smurf","sql injection","flooding","tampering","sanitize","sanitise", "unauthorized", "unauthorised"]

fixwords =["prevent", "fix", "attack", "protect", "issue", "correct", "update", "improve", "change", "check", "malicious", "insecure", "vulnerable", "vulnerability"]

#for all combinations of keywords and prefixes, scrape github for commits
for v in vulnerabilities:
  for fix in fixwords:
      getgitresponse(v + " " +fix,commits);

Streaming output truncated to the last 5000 lines.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
Rate limit. Sleep.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commits so far.
874 commit

In [ ]:
import requests
import time
import sys
import json
import os 

repositories = {}
data = {}


def gettoken():
  with open('/drive/MyDrive/token.txt', 'r') as token:
        gittoken = token.readline().replace("\n","")
  return gittoken

#get mined commits
repositories = {}
with open('/drive/MyDrive/commits.json', 'r') as infile:
    repositories = json.load(infile)

if os.path.isfile('/drive/MyDrive/DataFilter.json'):
  with open('/drive/MyDrive/DataFilter.json', 'r') as infile:
      data = json.load(infile)


if not "showcase" in data:
  data["showcase"] = {}
if not "noshowcase" in data:
  data["noshowcase"] = {}


toomuchsecurity = ['offensive', 'pentest', 'vulnerab', 'security', 'hack', 'exploit', 'ctf ', ' ctf', 'capture the flag','attack'] #keywords that are not allowed to appear in the repository name
alittletoomuch = ['offensive security', 'pentest', 'exploits', 'vulnerability research', 'hacking', 'security framework', 'vulnerability database', 'simulated attack', 'security research'] #keywords that are not allowed to appear in the readme description


myheaders = {'Authorization': 'token ' + gettoken()}

for repo in repositories:
    #get name of the repository
    name = repo.split('https://github.com/')[1]
    
    #if we don't know yet if it is a showcase...
    if (name in data['showcase']):
        continue
    if (name in data['noshowcase']):
        continue
    
    #check all the 'toomuch' keywords to see if they already appear in the name of the repository
    for toomuch in toomuchsecurity:
      if toomuch in name:
        #put it in the showcase categpry
        data['showcase'][name] = {}
        print(name + ": showcase")
        continue

    #get the readme of the repository
    response = requests.get('https://github.com/'+name+'/blob/master/README.md', headers = myheaders)        
    h = response.headers
            
    if ("markdown-body") in response.text:
      #find the description of the project
      pos = response.text.find("markdown-body")
      pos2 = response.text.find("/article")
      description = response.text[pos:pos2]
      
      #check all keywords from the 'alittletoomuch' category
      for toomuch in alittletoomuch: 
        if toomuch in description:
          #put it in the showcase category
          data['showcase'][name] = {}
          print(name + ": showcase")
          continue
      
print(name + ": not a showcase")
data['noshowcase'][name] = {}
      
with open('/drive/MyDrive/DataFilter.json', 'w') as outfile:
    json.dump(data, outfile)


KeyboardInterrupt: ignored

In [ ]:
import requests
import time
import sys
import json
import datetime



#get access token
# if not os.path.isfile('access'):
#   print("please place a Github access token in this directory.")
#   sys.exit()
# with open('access', 'r') as accestoken:
#   access = accestoken.readline().replace("\n","")

#get mined commits
repositories = {}
with open('/drive/MyDrive/commits.json', 'r') as infile:
    repositories = json.load(infile)
    

datafilter = {}

#load progress
if os.path.isfile('/drive/MyDrive/DataFilter.json'):
  with open('/drive/MyDrive/DataFilter.json', 'r') as infile:
      datafilter = json.load(infile)

if not 'showcase' in datafilter:
  datafilter['showcase'] = {}
if not 'no-python' in datafilter:
  datafilter['no-python'] = {}
if not 'python' in datafilter:
  datafilter['python'] = {}


print(str(len(datafilter['showcase'])) + " repositories are showcases and therefore ignored.")
print(str(len(datafilter['no-python'])) + " repositories don't even contain ANY python.")
print(str(len(datafilter['python'])) + " might contain python.")

data = {}

myheaders = {'Authorization': 'token ' + gettoken()}
progress = 0
total = 0
newrepos = 0
nopythonlist = {} #used to mark which commits have and don't have python files modified

#starting to collect requested data
for repo in repositories:
  progress = progress + 1

  if (((progress % 3000) == 0) and total > 0) and not saved:
    print("Time to save.")
    saved = True
    before = time.time()
    with open('DataFilter.json', 'w') as outfile:
        json.dump(datafilter, outfile)    
    with open('PyCommitsWithDiffs.json', 'w') as outfile:
        json.dump(data, outfile)

  name = repo.split('https://github.com/')[1]
  
  if (name in datafilter['showcase']):
      print("skip: showcase")
      continue
  if (name in datafilter['no-python']):
      print("skip: no python")
      continue

  print("\n" + repo + "     " + str(progress))

  if not repo in nopythonlist:
    nopythonlist[repo] = {}
    
  noPythonAtAll = True
  
  for c in repositories[repo]:
      #go through all commits of that repository
      
      if c in nopythonlist[repo]:
        #if we already know that the commit has no python, skip it
        continue
      
      #otherwise, get the DIFF file
      target = repo+'/commit/' + c + '.diff'
      response = requests.get(target,headers = myheaders)
      #this is the diff
      content = response.content
      try:
          diffcontent = content.decode('utf-8',errors='ignore');
      except:
          print("an exception occured. Skip.");
          continue;
      
      #check if the file contains any python
      if (".py" in diffcontent):        
          noPythonAtAll = False #the repository in general contains at least some python 
          
          #put it in data
          if not repo in data:
            data[repo] = {} 
            
          #we should save again when the time is right
          total = total + 1 
          saved = False
          
          #copy the relevant information to 'data'
          data[repo][c] = repositories[repo][c]
          data[repo][c]["diff"] = content.decode('utf-8',errors='ignore');

      else:
          if not repo in nopythonlist:
            nopythonlist[repo] = {}
          #note down that this commit doesn't contain any pyhon files
          nopythonlist[repo][c] = {}

        
  if noPythonAtAll:
    #note down that this repository doesn't contain any python files
    datafilter['no-python'][name] = {}    
  else:
    #repository has some python, and we checked it now
    datafilter['python'][name] = {}



print(str(total) + " commits modifying python were found.")

#save the markings of what contains no python data
with open('/drive/MyDrive/DataFilter.json', 'w') as outfile:
    json.dump(datafilter, outfile)

#save the actual python results
with open('/drive/MyDrive/PyCommitsWithDiffs.json', 'w') as outfile:
    json.dump(data, outfile)
    

9 repositories are showcases and therefore ignored.
0 repositories don't even contain ANY python.
0 might contain python.

https://github.com/ipfs/go-graphsync     1

https://github.com/TALUAtGitHub/android_kernel_samsung_exynos7870     2

https://github.com/dotnet/dotNext     3

https://github.com/Jasmine-Wayne-4-19/android_kernel_xiaomi_sdm660_4.19     4

https://github.com/AOSPK-Devices/kernel_xiaomi_sm6250     5

https://github.com/AICP/kernel_xiaomi_vayu     6

https://github.com/Sleppyiecat/just_love_me     7

https://github.com/OpenModelica/OpenModelica     8

https://github.com/ccns/dreambbs     9

https://github.com/aligrudi/neatpost     10

https://github.com/Pfannex/OmniESP     11

https://github.com/BigMumf/display-keyboard     12

https://github.com/Seagate/openSeaChest     13

https://github.com/tetcoin/tetcore     14

https://github.com/Robbbert/messui     15

https://github.com/Robbbert/mameui     16

https://github.com/Robbbert/store1     17

https://github.com/Robbber

In [ ]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model
from keras.layers.embeddings import Embedding
from keras.layers import Bidirectional
from keras.preprocessing import sequence
from keras import backend as K
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.utils import class_weight
import tensorflow as tf
import builtins
import keyword
import pickle
import numpy
import os
from gensim.models import Word2Vec, KeyedVectors
import tensorflow as tf
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from termcolor import colored


  

def findComments(sourcecode):
  commentareas = []
  
  inacomment = False
  commentstart = -1
  commentend = -1
  
  
  for pos in range(len(sourcecode)):
    if sourcecode[pos] == "#":
      if not inacomment:
        commentstart = pos 
        inacomment = True
    
    if sourcecode[pos] == "\n":
      if inacomment:
        commentend = pos
        inacomment = False
    
    if commentstart >= 0 and commentend >= 0:
      t = [commentstart, commentend]
      commentareas.append(t)
      commentstart = -1
      commentend = -1


  return commentareas


def findposition(badpart,sourcecode):
  splitchars = ["\t", "\n", " ", ".", ":", "(", ")", "[", "]", "<", ">", "+", "-", "=","\"", "\'","*", "/","\\","~","{","}","!","?","*",";",",","%","&"]
  pos = 0
  matchindex = 0
  inacomment = False
  bigcomment = False
  bigcomment2 = False
  startfound = -1
  endfound = -1
  position = []
  end = False
  last = 0
  
  while "#" in badpart:
    f = badpart.find("#")
    badpart = badpart[:f]

  b = badpart.lstrip()
  if len(b) < 1:
  #  print(b)
  #  print("nope.\n\n")
    return[-1,-1]


  while(not end):
    #print("position : " + str(pos))
    
    if not inacomment:
      last = pos-1
    
    if pos >= len(sourcecode):
      end = True
      break
    
    if sourcecode[pos] == "\n":
 #     print("end of comment")
 #     print("[" + sourcecode[last]+ "]")
      inacomment = False
      
    if sourcecode[pos] == "\n" and (sourcecode[pos-1] == "\n" or sourcecode[last] == " "):
      #print("one further")
      pos = pos +1
      continue
      
    if sourcecode[pos] == " " and (sourcecode[pos-1] == " " or sourcecode[last] == "\n"):
     # print("one further")
      pos = pos +1
      continue
      
    if sourcecode[pos] == "#":
      
      inacomment = True
      

#    if sourcecode[pos] == "'":      
#      if pos+2 < len(sourcecode):
#        if sourcecode[pos+1] == "'" and sourcecode[pos+2] == "'":
#          if not bigcomment:
#            pos = pos+3
#            bigcomment = True
#        #    print(">>> BIGCOMMENT")
#            continue
#          else:
#            pos = pos+3
#            bigcomment = False
#         #   print(">>> BIGCOMMENT END")
#            continue

#    if sourcecode[pos] == '"':      
#      if pos+2 < len(sourcecode):
#        if sourcecode[pos+1] == '"' and sourcecode[pos+2] == '"':
#          if not bigcomment2:
#            pos = pos+3
#            bigcomment2 = True
#          #  print(">>> BIGCOMMENT")
#            continue
#          else:
#            pos = pos+3
#            bigcomment2 = False
#           # print(">>> BIGCOMMENT END")
#            continue
      
    if (False):
      
                      print("---------------------------------")
                      string1 = ""
                      string2 = ""
                      for i in range(0,pos):
                        string1 = string1 + sourcecode[i]

                      for i in range(pos+1,len(sourcecode)):
                        string2 = string2 + sourcecode[i]
                        
                      print(string1 + "[" + sourcecode[pos] + "]" + string2)
                      print("---------------------------------")


                      string1 = ""
                      string2 = ""
                      
                      for i in range(0,matchindex):
                        string1 = string1 + badpart[i]

                      for i in range(matchindex+1,len(badpart)):
                        string2 = string2 + badpart[i]
                        
                      print(string1 + "[" + badpart[matchindex] + "]" + string2)
  
                      print("---------------------------------")
                

    if not inacomment: # and not bigcomment and not bigcomment2:
      a = sourcecode[pos]
      if a == "\n":
        a = " "
      b = badpart[matchindex]
      
      c = ""
      if matchindex > 0:
        c = badpart[matchindex-1]
      
      d = ""
      if matchindex < len(badpart)-2:
        d = badpart[matchindex+1]
        
      if (a != b) and (a == " " or a == "\n") and ((b in splitchars) or (c in splitchars)):
        pos = pos+1
        continue
      
      if (a != b) and (b == " " or b == "\n"):
        #print("here")
        if (c in splitchars or d in splitchars):
          #print("here2")
          if (matchindex < len(badpart)-1):
            matchindex = matchindex + 1
            continue
        
      if a == b:
          if matchindex == 0:
            startfound = pos
         # print("\n>>match: " + badpart[matchindex] + "(" + str(matchindex) + "/" + str(len(badpart)) + ")\n\n")
          matchindex = matchindex + 1
          
      else:
          #print("\n>>no match" )
          matchindex = 0
          startfound = -1
        
      if matchindex == len(badpart):
        endfound = pos
    #    print("FOUND at pos "+ str(startfound) + ":" + str(endfound))
        break
        
    if pos == len(sourcecode):
      end = True
    pos = pos + 1
  
  position.append(startfound)
  position.append(endfound)
  
  if endfound < 0:
    startfound = -1
    
  if endfound < 0 and startfound < 0: #and not "#" in badpart and not '"""' in badpart and not "'''" in badpart:
#    print(sourcecode)
#    print(":::::::::::")
#    print(badpart)
#    print("-----------------")
    return[-1,-1]
  return position




def findpositions(badparts,sourcecode):
  
  positions = []
  
  
  for bad in badparts:
    
    if "#" in bad:
      find = bad.find("#")
      bad = bad[:find]
      
    place = findposition(bad,sourcecode)
    if place != [-1,-1]:
      positions.append(place)
    
    
  return positions
  








def nextsplit(sourcecode,focus):
  splitchars = [" ","\t","\n", ".", ":", "(", ")", "[", "]", "<", ">", "+", "-", "=","\"", "\'","*", "/","\\","~","{","}","!","?","*",";",",","%","&"]
  for pos in range(focus+1, len(sourcecode)):
      if sourcecode[pos] in splitchars:
        return pos
  return -1

def previoussplit(sourcecode,focus):
  splitchars = [" ","\t","\n", ".", ":", "(", ")", "[", "]", "<", ">", "+", "-", "=","\"", "\'","*", "/","\\","~","{","}","!","?","*",";",",","%","&"]
  pos = focus-1
  while(pos >= 0):
      if sourcecode[pos] in splitchars:
        return pos
      pos = pos-1
  return -1

def getcontextPos(sourcecode,focus,fulllength):

  
  startcontext = focus
  endcontext = focus
  if focus > len(sourcecode)-1:
    return None

  start = True
  
      
  while not len(sourcecode[startcontext:endcontext]) > fulllength:
   # print(str(startcontext) + ":" + str(endcontext))
   # print(len(sourcecode[startcontext:endcontext]))
    
    if previoussplit(sourcecode,startcontext) == -1 and nextsplit(sourcecode,endcontext) == -1:
   #   print("NONE!")
      return None
    
    if start:
      if previoussplit(sourcecode,startcontext) > -1:
        startcontext = previoussplit(sourcecode,startcontext)
      #print("new start: " + str(startcontext))
      start = False
    else:
      if nextsplit(sourcecode,endcontext) > -1:
        endcontext = nextsplit(sourcecode,endcontext)
      #print("new end: " + str(endcontext))
      start = True

        
#  print("focus: " + str(focus))
#  print("start: " + str(startcontext))
#  print("end: " + str(endcontext))
  return [startcontext,endcontext]

def getcontext(sourcecode,focus,fulllength):

  
  startcontext = focus
  endcontext = focus
  if focus > len(sourcecode)-1:
    return None

  start = True
  
      
  while not len(sourcecode[startcontext:endcontext]) > fulllength:
   # print(str(startcontext) + ":" + str(endcontext))
   # print(len(sourcecode[startcontext:endcontext]))
    
    if previoussplit(sourcecode,startcontext) == -1 and nextsplit(sourcecode,endcontext) == -1:
   #   print("NONE!")
      return None
    
    if start:
      if previoussplit(sourcecode,startcontext) > -1:
        startcontext = previoussplit(sourcecode,startcontext)
      #print("new start: " + str(startcontext))
      start = False
    else:
      if nextsplit(sourcecode,endcontext) > -1:
        endcontext = nextsplit(sourcecode,endcontext)
      #print("new end: " + str(endcontext))
      start = True

        
#  print("focus: " + str(focus))
#  print("start: " + str(startcontext))
#  print("end: " + str(endcontext))
  return sourcecode[startcontext:endcontext]
  











def getblocks(sourcecode, badpositions, step, fulllength):
      blocks = []
       
      focus = 0
      lastfocus = 0
      while (True):
        if focus > len(sourcecode):
          break
        
        focusarea = sourcecode[lastfocus:focus]
                
        if not (focusarea == "\n"):
              
            middle = lastfocus+round(0.5*(focus-lastfocus))              
            context = getcontextPos(sourcecode,middle,fulllength)
            #print([lastfocus,focus,len(sourcecode)])
            
            
            if context is not None:
              
              
                
              vulnerablePos = False
              for bad in badpositions:
                    
                  if (context[0] > bad[0] and context[0] <= bad[1]) or (context[1] > bad[0] and context[1] <= bad[1]) or (context[0] <= bad[0] and context[1] >= bad[1]):
                    vulnerablePos = True
            
                  
              q = -1
              if vulnerablePos:
                q = 0
              else:
                q = 1
              
              
              singleblock = []
              singleblock.append(sourcecode[context[0]:context[1]])
              singleblock.append(q)
                
              already = False
              for b in blocks:
                if b[0] == singleblock[0]:
                #  print("already.")
                  already = True
                  
              if not already:
                blocks.append(singleblock)


        if ("\n" in sourcecode[focus+1:focus+7]):
          lastfocus = focus
          focus = focus + sourcecode[focus+1:focus+7].find("\n")+1
        else:
          if nextsplit(sourcecode,focus+step) > -1:
            lastfocus = focus
            focus = nextsplit(sourcecode,focus+step)
          else:
            if focus < len(sourcecode):
              lastfocus = focus
              focus = len(sourcecode)
            else:
              break

      
      return blocks




def getBadpart(change):
  
  #print("\n")
  removal = False
  lines = change.split("\n")
  for l in lines:
    if len(l) > 0:
      if l[0] == "-":
        #print("a line is removed")
        removal = True
      
  
  if not removal:
    #print("There is no removal.")
    return None
  
  pairs = []
  
  badexamples = []
  goodexamples = []

  for l in range(len(lines)):
    
    line = lines[l]
    line = line.lstrip()
    if len(line.replace(" ","")) > 1:
        if line[0] == "-":
          if not "#" in line[1:].lstrip()[:3] and not "import os" in line:
            badexamples.append(line[1:])
        if line[0] == "+":
          if not "#" in line[1:].lstrip()[:3] and not "import os" in line:
            goodexamples.append(line[1:])
    
  if len(badexamples) == 0:
#    print("removed lines were empty or comments")
    return None
  
  return [badexamples,goodexamples]
    


  

def getTokens(change):
  tokens = []  
  
  change = change.replace(" .",".")
  change = change.replace(" ,",",")
  change = change.replace(" )",")")
  change = change.replace(" (","(")
  change = change.replace(" ]","]")
  change = change.replace(" [","[")
  change = change.replace(" {","{")
  change = change.replace(" }","}")
  change = change.replace(" :",":")
  change = change.replace("- ","-")
  change = change.replace("+ ","+")
  change = change.replace(" =","=")
  change = change.replace("= ","=")
  splitchars = [" ","\t","\n", ".", ":", "(", ")", "[", "]", "<", ">", "+", "-", "=","\"", "\'","*", "/","\\","~","{","}","!","?","*",";",",","%","&"]
  start = 0
  end = 0
  for i in range(0, len(change)):
    if change[i] in splitchars:
      if i > start:
        start = start+1
        end = i
        if start == 1:
          token = change[:end]
        else:
          token = change[start:end]
        if len(token) > 0:
          tokens.append(token)
        tokens.append(change[i])
        start = i
  return(tokens)
  



def removeDoubleSeperatorsString(string):
  return ("".join(removeDoubleSeperators(getTokens(string))))







def removeDoubleSeperators(tokenlist):
    last = ""
    newtokens = []
    for token in tokenlist:
      if token == "\n":
        token = " "
      if len(token) > 0:
        if ((last == " ") and (token == " ")):
          o = 1 #noop
          #print("too many \\n.")
        else:
          newtokens.append(token)
          
        last = token
        
    return(newtokens)
  
  
def isEmpty(code):
  token = getTokens(stripComments(code))
  for t in token:
    if (t != "\n" and t != " "):
      return False
  return True

def is_builtin(name):
    return name in builtins.__dict__
def is_keyword(name):
      return name in keyword.kwlist



def removeTripleN(tokenlist):
    secondlast = ""
    last = ""
    newtokens = []
    for token in tokenlist:
      if len(token) > 0:
        
        if ((secondlast == "\n") and (last == "\n") and (token == "\n")):
          #print("too many \\n.")
          o = 1 #noop
        else:
          newtokens.append(token)
          
        
        thirdlast = secondlast
        secondlast = last
        last = token
        
    return(newtokens)
  
  



def getgoodblocks(sourcecode,goodpositions,fullength):
  blocks = []
  if (len(goodpositions) > 0):
    for g in goodpositions:
     # print("g " + str(g))
      if g != []:
        focus = g[0]
        while (True):
          if focus >= g[1]:
            #print("  too far.")
            break

    #        print("Focus is on " + str(focus) + " " + sourcecode[focus])
            
          
          context = getcontext(sourcecode,focus,fulllength)
          
          if context is not None:
            singleblock = []
            singleblock.append(context)
            singleblock.append(1)
              
            already = False
            for b in blocks:
              if b[0] == singleblock[0]:
              #  print("already.")
                already = True
                  
            if not already:
              blocks.append(singleblock)
              
              
            if nextsplit(sourcecode,focus+15) > -1:
              focus = nextsplit(sourcecode,focus+15)
            else:
              break
      
#  if len(blocks) > 0:
#    print(blocks)
  return blocks




def stripComments(code):
    
  withoutComments = ""
  lines = code.split("\n")
  withoutComments = ""
  therewasacomment = False
  for c in lines:
    if "#" in c:
      therewasacomment = True
      position = c.find("#")
      c = c[:position]
    withoutComments = withoutComments + c + "\n"
  
  
  change = withoutComments
  
  withoutComments = change

  return withoutComments





#Define F1 loss and measurement

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)
  
  
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))



def predict(vectorlist,model): 
  
  if (len(vectorlist) > 0):
    one = []
    one.append(vectorlist)
    one = numpy.array(one)
    max_length = 200
    one = sequence.pad_sequences(one, maxlen=max_length)
    yhat_probs = model.predict(one, verbose=0)
    prediction = int(yhat_probs[0][0] * 100000)
    prediction = 0.00001 * prediction 
    return prediction
    
  else:
    return -1




def getblocksVisual(mode,sourcecode, badpositions,commentareas, fulllength,step, nr,w2v_model,model,threshold,name):

      word_vectors = w2v_model.wv
      
      ypos = 0
      xpos = 0
      
      lines = (sourcecode.count("\n"))
      #print("lines: " + str(lines))
      img = Image.new('RGBA', (2000, 11*(lines+1)))
      color = "white"
      
      blocks = []
       
      focus = 0
      lastfocus = 0
      
      string = ""
      
      trueP = False
      falseP = False
      
      while (True):
        if focus > len(sourcecode):
          break
        
        
        
        comment = False
        for com in commentareas:
          
          if (focus >= com[0] and focus <= com[1] and lastfocus >= com[0] and lastfocus < com[1]):
                focus = com[1]
                #print("within")
                comment = True
          if (focus > com[0] and focus <= com[1] and  lastfocus < com[0]):
              focus = com[0]
              #print("before")
              comment = False                   
          elif (lastfocus >= com[0] and lastfocus < com[1] and focus > com[1]):
              focus = com[1]
              #print("up to the end")
              comment = True
      
        #print([lastfocus,focus,comment, "["+sourcecode[lastfocus:focus]+"]"])
        focusarea = sourcecode[lastfocus:focus]
     
        if(focusarea == "\n"):
          string = string + "\n"
          
        else:
          if comment:
              color = "grey"
              string = string + colored(focusarea,'grey')
          else:
              
              
              middle = lastfocus+round(0.5*(focus-lastfocus))              
              context = getcontextPos(sourcecode,middle,fulllength)
              
              
              if context is not None:
              
                vulnerablePos = False
                for bad in badpositions:
                    if (context[0] > bad[0] and context[0] <= bad[1]) or (context[1] > bad[0] and context[1] <= bad[1]) or (context[0] <= bad[0] and context[1] >= bad[1]):
                      vulnerablePos = True
                      
                predictionWasMade = False
                text = sourcecode[context[0]:context[1]].replace("\n", " ")
                token = getTokens(text)
                if (len(token) > 1):                  
                  vectorlist = []                  
                  for t in token:
                    if t in word_vectors.vocab and t != " ":
                      vector = w2v_model[t]
                      vectorlist.append(vector.tolist())   
                      
                  if len(vectorlist) > 0:
                      p = predict(vectorlist,model)
                      if p >= 0:
                        predictionWasMade = True
                        
                      #  print(p)
                        if vulnerablePos:
                          if p > 0.5:
                            color = "royalblue"
                            string = string + colored(focusarea,'cyan')
                          else:
                            string = string + colored(focusarea,'magenta')
                            color = "violet"
                            
                        else:
                          
                        
                          if p > threshold[0]:
                            color = "darkred"
                          elif p >  threshold[1]:
                            color = "red"
                          elif p >  threshold[2]:
                            color = "darkorange"
                          elif p >  threshold[3]:
                            color = "orange"
                          elif p >  threshold[4]:
                            color = "gold"
                          elif p >  threshold[5]:
                            color = "yellow"
                          elif p >  threshold[6]:
                            color = "GreenYellow"
                          elif p >  threshold[7]:
                            color = "LimeGreen"
                          elif p >  threshold[8]:
                            color = "Green"
                          else:
                            color = "DarkGreen"
                
                          if p > 0.8:
                            string = string + colored(focusarea,'red')
                          elif p > 0.5:
                            string = string + colored(focusarea,'yellow')
                          else:
                            string = string + colored(focusarea,'green')
                            
                if not predictionWasMade:
                  string = string + focusarea
              else:
                string = string + focusarea
                
        
        
            
        try:
          if len(focusarea) > 0:
            d = ImageDraw.Draw(img)
#            print(list(focusarea))
            if focusarea[0] == "\n":
              ypos = ypos + 11
              xpos = 0
              d.text((xpos, ypos), focusarea[1:], fill=color)
              xpos = xpos + d.textsize(focusarea)[0]
            else:
              d.text((xpos, ypos), focusarea, fill=color)
              xpos = xpos + d.textsize(focusarea)[0]

        except Exception as e:
          print(e)

        if ("\n" in sourcecode[focus+1:focus+7]):
          lastfocus = focus
          focus = focus + sourcecode[focus+1:focus+7].find("\n")+1
        else:
          if nextsplit(sourcecode,focus+step) > -1:
            lastfocus = focus
            focus = nextsplit(sourcecode,focus+step)
          else:
            if focus < len(sourcecode):
              lastfocus = focus
              focus = len(sourcecode)
            else:
              break

      
      
      #print(string)
      
      for i in range(1,100):
        if not os.path.isfile('demo_' + mode + "_" + str(i) +"_"+ name + '.png'):
                img.save('demo_' + mode + "_" + str(i) + "_" + name + '.png')    
                print("saved png.")
                break
      return blocks



def getIdentifiers(mode,nr):
  #print("getting " + mode + " " + nr)
  if mode == "sql":
    if nr == "1":
      rep = "instacart/lore"
      com = "a0a5fd945a8bf128d4b9fb6a3ebc6306f82fa4d0"
      myfile = "/lore/io/connection.py"
    elif nr == "2":
      rep = "uktrade/export-wins-data"
      com = "307587cc00d2290a433bf74bd305aecffcbb05a2"
      myfile = "/wins/views/flat_csv.py"
    elif nr == "3":
      rep = "onewyoming/onewyoming"
      com = "54fc7b076fda2de74eeb55e6b75b28e09ef231c2"
      myfile = "/experimental/python/buford/model/visitor.py"
  if mode == "xss":
    if nr == "1":
      rep = "AMfalme/Horizon_Openstack"
      com = "a835dbfbaa2c70329c08d4b8429d49315dc6d651"
      myfile = "/openstack_dashboard/dashboards/identity/mappings/tables.py"
    elif nr == "2":
      rep = "omirajkar/bench_frappe"
      com = "2fa19c25066ed17478d683666895e3266936aee6"
      myfile = "/frappe/website/doctype/blog_post/blog_post.py"
    elif nr == "3":
      rep = "Technikradio/C3FOCSite"
      com = "6e330d4d44bbfdfce9993dffea97008276771600"
      myfile = "/c3shop/frontpage/management/reservation_actions.py"
  if mode == "command_injection":
    if nr == "1":
      rep = "dgabbe/os-x"
      com = "bb2ded2dbbbac8966a77cc8aa227011a8b8772c0"
      myfile = "/os-x-config/standard_tweaks/install_mac_tweaks.py"
    elif nr == "2":
      rep = "Atticuss/ajar"
      com = "5ed8aba271ad20e6168f2e3bd6c25ba89b84484f"
      myfile = "/ajar.py"
    elif nr == "3":
      rep = "yasong/netzob"
      com = "557abf64867d715497979b029efedbd2777b912e"
      myfile = "/src/netzob/Simulator/Channels/RawEthernetClient.py"
  if mode == "xsrf":
    if nr == "1":
      rep = "deepnote/notebook"
      com = "d7becafd593c2958d8a241928412ddf4ba801a42"
      myfile = "/notebook/files/handlers.py"
    elif nr == "2":
      rep = "wbrxcorp/forgetthespiltmilk"
      com = "51bed3f7f01079d91864ddc386a73eb3e1ca634b"
      myfile = "/frontend/app.py"
    elif nr == "3":
      rep = "tricycle/lesswrong"
      com = "ef303fe078c60d964e3f9e87d3da1a67fecd2c2b"
      myfile = "/r2/r2/models/account.py"
  if mode == "remote_code_execution":
    if nr == "1":
      rep = "Internet-of-People/titania-os"
      com = "9b7805119938343fcac9dc929d8882f1d97cf14a"
      myfile = "/vuedj/configtitania/views.py"
    elif nr == "2":
      rep = "Scout24/monitoring-config-generator"
      com = "2191fe6c5a850ddcf7a78f7913881cef1677500d"
      myfile = "/src/main/python/monitoring_config_generator/yaml_tools/readers.py"
    elif nr == "3":
      rep = "pipermerriam/flex"
      com = "329c0a8ae6fde575a7d9077f1013fa4a86112d0c"
      myfile = "/flex/core.py"
  if mode == "path_disclosure":
    if nr == "1":
      rep = "fkmclane/python-fooster-web"
      com = "80202a6d3788ad1212a162d19785c600025e6aa4"
      myfile = "/fooster/web/file.py"
    elif nr == "2":
      rep = "zms-publishing/zms4"
      com = "3f28620d475220dfdb06f79787158ac50727c61a"
      myfile = "/ZMSItem.py"
    elif nr == "3":
      rep = "cuckoosandbox/cuckoo"
      com = "168cabf86730d56b7fa319278bf0f0034052666a"
      myfile = "/cuckoo/web/controllers/submission/api.py"
  if mode == "open_redirect":
      if nr == "1":
        rep = "karambir/mozilla-django-oidc"
        com = "22b6ecb953bbf40f0394a8bfd41d71a3f16e3465"
        myfile = "/mozilla_django_oidc/views.py"
      elif nr == "2":
        rep = "nyaadevs/nyaa"
        com = "b2ddba994ca5e78fa5dcbc0e00d6171a44b0b338"
        myfile = "/nyaa/views/account.py"
      elif nr == "3":
        rep = "richgieg/flask-now"
        com = "03df8ce6bddc56b2487df3898758f4c1624d906f"
        myfile = "/app/auth/views.py"


  
  result = []
  result.append(rep)
  result.append(com)
  result.append(myfile)
  return result
  



def getFromDataset(identifying,data):
  
  result = []
  rep = identifying[0]
  com = identifying[1]
  myfile = identifying[2]
  #print("\n")
  #print("getting from dataset: " + rep + "/commit/" + com + " " + myfile)
  repfound = False
  comfound = False
  filefound = False
  for r in data:
        if  "https://github.com/"+rep ==r:
         #   print("  found repository.")
            repfound = True
            for c in data[r]:
              if c == com:
          #      print("    found commit.")
                comfound = True
                if "files" in data[r][c]:
                    for f in data[r][c]["files"].keys():
                      if myfile == f:  
                          filefound = True
           #               print("      found file")
                          if "source" in data[r][c]["files"][f]:                          
                              allbadparts = []
                              sourcecode = data[r][c]["files"][f]["source"]
                              sourcefull = data[r][c]["files"][f]["sourceWithComments"]
                              
                              
                              for change in data[r][c]["files"][f]["changes"]:
                                badparts = change["badparts"]
                                
                                if (len(badparts) < 20):
                                  for bad in badparts:
                                    pos = findposition(bad,sourcecode)
                                    if not -1 in pos:
                                      allbadparts.append(bad)
                                      
                                                                  
                              result.append(sourcefull)
                              result.append(allbadparts)
                              
                              if not repfound:
                                print("Rep found " + str(repfound))
                              elif not comfound:
                                print("Com found " + str(comfound))
                              elif not filefound:
                                print("File found " + str(filefound))
                              return(result)
                        

  if not repfound:
    print("Rep found " + str(repfound))
  elif not comfound:
    print("Com found " + str(comfound))
  elif not filefound:
    print("File found " + str(filefound))
  return []

In [ ]:
import requests
import time
import sys
import json
import datetime
import os


#get access token
def gettoken():
  with open('/drive/MyDrive/token.txt', 'r') as token:
        gittoken = token.readline().replace("\n","")
  return gittoken


#get mined commits
repositories = {}
with open('/drive/MyDrive/commits.json', 'r') as infile:
    repositories = json.load(infile)
    

datafilter = {}

#load progress
if os.path.isfile('/drive/MyDrive/DataFilter.json'):
  with open('/drive/MyDrive/DataFilter.json', 'r') as infile:
      datafilter = json.load(infile)

if not 'showcase' in datafilter:
  datafilter['showcase'] = {}
if not 'no-python' in datafilter:
  datafilter['no-python'] = {}
if not 'python' in datafilter:
  datafilter['python'] = {}


print(str(len(datafilter['showcase'])) + " repositories are showcases and therefore ignored.")
print(str(len(datafilter['no-python'])) + " repositories don't even contain ANY python.")
print(str(len(datafilter['python'])) + " might contain python.")

data = {}

myheaders = {'Authorization': 'token ' + gettoken()}
progress = 0
total = 0
newrepos = 0
nopythonlist = {} #used to mark which commits have and don't have python files modified

#starting to collect requested data
for repo in repositories:
  progress = progress + 1

  if (((progress % 3000) == 0) and total > 0) and not saved:
    print("Time to save.")
    saved = True
    before = time.time()
    with open('DataFilter.json', 'w') as outfile:
        json.dump(datafilter, outfile)    
    with open('PyCommitsWithDiffs.json', 'w') as outfile:
        json.dump(data, outfile)

  name = repo.split('https://github.com/')[1]
  
  if (name in datafilter['showcase']):
      print("skip: showcase")
      continue
  if (name in datafilter['no-python']):
      print("skip: no python")
      continue

  print("\n" + repo + "     " + str(progress))

  if not repo in nopythonlist:
    nopythonlist[repo] = {}
    
  noPythonAtAll = True
  
  for c in repositories[repo]:
      #go through all commits of that repository
      
      if c in nopythonlist[repo]:
        #if we already know that the commit has no python, skip it
        continue
      
      #otherwise, get the DIFF file
      target = repo+'/commit/' + c + '.diff'
      response = requests.get(target,headers = myheaders)
      #this is the diff
      content = response.content
      try:
          diffcontent = content.decode('utf-8',errors='ignore');
      except:
          print("an exception occured. Skip.");
          continue;
      
      #check if the file contains any python
      if (".py" in diffcontent):        
          noPythonAtAll = False #the repository in general contains at least some python 
          
          #put it in data
          if not repo in data:
            data[repo] = {} 
            
          #we should save again when the time is right
          total = total + 1 
          saved = False
          
          #copy the relevant information to 'data'
          data[repo][c] = repositories[repo][c]
          data[repo][c]["diff"] = content.decode('utf-8',errors='ignore');

      else:
          if not repo in nopythonlist:
            nopythonlist[repo] = {}
          #note down that this commit doesn't contain any pyhon files
          nopythonlist[repo][c] = {}

        
  if noPythonAtAll:
    #note down that this repository doesn't contain any python files
    datafilter['no-python'][name] = {}    
  else:
    #repository has some python, and we checked it now
    datafilter['python'][name] = {}



print(str(total) + " commits modifying python were found.")

#save the markings of what contains no python data
with open('/drive/MyDrive/DataFilter.json', 'w') as outfile:
    json.dump(datafilter, outfile)

#save the actual python results
with open('/drive/MyDrive/PyCommitsWithDiffs.json', 'w') as outfile:
    json.dump(data, outfile)
    

OSError: ignored

In [ ]:
import time
import sys
import json
import subprocess
from datetime import datetime
import requests 
import pickle
from pydriller import Repository

def getDiffs(rest):
  ### get the changes from the .diff file
  diffs = []
  while("diff --git" in rest):
    file = ""
    start = rest.find("diff --git")+1
    nextindex = rest.find("index")+1
    #get filenames line
    filenameline = rest[start:nextindex]
    if not (".py") in rest[start:secondpart]:
      # No python file changed in this part of the commit
      rest = rest[secondpart+1]
      continue

    if "diff --git" in rest[start:]:
      end = rest[start:].find("diff --git");
      filechange = rest[start:end]
      rest = rest[end:]
    else:
      end = len(rest)
      filechange = rest[start:end]
      rest = ""
    filechangerest = filechange

    while ("@@" in filechangerest):
      ### extract all singular changes, which are recognizable by the @@ marking the line number
      change = ""
      start = filechangerest.find("@@")+2
      start2 = filechangerest[start:start+50].find("@@")+2
      start = start+start2
      filechangerest=filechangerest[start:]

      if ("class" in filechangerest or "def" in filechangerest) and "\n" in filechangerest:
        filechangerest = filechangerest[filechangerest.find("\n"):]
      
      if "@@" in filechangerest:
          end = filechangerest.find("@@")
          change = filechangerest[:end]
          filechangerest = filechangerest[end+2:]

      else:
        end = len(filechangerest)
        change = filechangerest[:end]
        filechangerest = ""
        
      if len(change) > 0:
        changes.append([titleline,change])
  
  return changes


def getfilename(line):
  #extracts the file name from the title line of a diff file
  s = line.find(" a/")+2
  e = line.find(" b/")
  name = line[s:e]
  
  if leline.count(name) == 2:
    return name
  elif ".py" in name and (" a"+name+" " in line):
    return name
  else:
    print("couldn't find name")
    print(line)
    print(name)
  
def makechangeobj(changeobj):
  #for a single change, consisting of titleline and raw code, create a usable object by extracting all relevant info
  
  change = changeobj[1]
  titleline = changeobj[0]
  
  if "<html" in change:
    return None
  
  if "sage:" in change or "sage :" in change:
    return None
  
  thischange = {}

  if myutils.getBadpart(change) is not None:      
    badparts = myutils.getBadpart(change)[0]
    goodparts = myutils.getBadpart(change)[1]
    linesadded = change.count("\n+")
    linesremoved = change.count("\n-")
    thischange["diff"] = change
    thischange["add"] = linesadded
    thischange["remove"] = linesremoved
    thischange["filename"] = getFilename(titleline)
    thischange["badparts"] = badparts
    thischange["goodparts"] = []
    if goodparts is not None:
      thischange["goodparts"] = goodparts
    if thischange["filename"] is not None:
      return thischange

  return None

#===========================================================================

with open('/content/drive/MyDrive/PyCommitsWithDiffs.json','r') as inputfile:
  data = json.load(inputfile)

now = datetime.now() # current date and time
nowformat = now.strftime("%H:%M")
print("finished loading ", nowformat)

progress = 0
changedict = {}

mode = "sql" 
for mode in ["remote_code_execution","redirect"]:
  for mode in ["remote_code_execution","redirect"]:
  
    if mode == "function_injection":
      allowedKeywords = ["function injection"]
    if mode == "remote_code_execution":
      allowedKeywords = ["remote code"]
    if mode == "cross_frame_scripting":
      allowedKeywords = ["cross frame"]
    if mode == "csv_injection":
      allowedKeywords = ["csv"]
    if mode == "redirect":
      allowedKeywords = ["redirect"]
    if mode == "hijack":
      allowedKeywords = ["session hijack","session fixation"]
    if mode == "command_injection":
      allowedKeywords = ["command injection"] 
    if mode == "sql":
      allowedKeywords = ["sql"] 
    if mode == "xsrf":
      allowedKeywords = ["xsrf","request forgery"] 
    if mode == "xss":
      allowedKeywords = ["xss", "cross site scripting","cross-site scripting"] 
    if mode == "replay_attack":
      allowedKeywords = ["replay attack"] 
    if mode == "unauthorized":
      allowedKeywords = ["unauthorized", "unauthorised"]
    if mode == "brute_force":
      allowedKeywords = ["brute force"]
    if mode == "flooding":
      allowedKeywords = ["flooding"]
    if mode == "remote_code_execution":
      allowedKeywords = ["remote code execution"]
    if mode == "formatstring":
      allowedKeywords = ["format string", "formatstring"]
    if mode == "session_fixation":
      allowedKeywords = ["fixation"]
    if mode == "cross_origin":
      allowedKeywords = ["cross origin"]
    if mode == "buffer overflow":
      allowedKeywords = ["buffer"]
    if mode == "cache":
      allowedKeywords = ["cache"]
    if mode == "eval":
      allowedKeywords = ["eval"]
    if mode == "csv":
      allowedKeywords = ["csv"]
    if mode == "path_disclosure":
      allowedKeywords = ["path"]
    if mode == "man-in-the-middle":
      allowedKeywords = ["man in the middle", "man-in-the-middle"]
    if mode == "smurf":
      allowedKeywords = ["smurf"]
    if mode == "tampering":
      allowedKeywords = ["tamper"]
    if mode == "sanitize":
      allowedKeywords = ["saniti"]
    if mode == "denial":
      allowedKeywords = ["denial"]
    if mode =="directory_traversal":
      allowedKeywords = ["directory","traversal"]
    if mode == "clickjack":
      allowedKeywords = ["clickjack", "click jack"]
    if allowedKeywords == "spoof":
      allowedKeywords = ["spoof"]


    suspiciouswords = ["injection", "vulnerability", "exploit", " ctf","capture the flag","ctf","burp","capture","flag","attack","hack"]

    #words that should not appear in the commit message
    badwords = ["sqlmap", "sql-map", "sql_map","ctf "," ctf"]



datanew = {}

for r in data:

   progress = progress +1
   suspicious = False
   for b in badwords:
        if b.lower() in r.lower():
          suspicious = True
   if suspicious:
        continue


   changesfromdiff = False
   all_irrelevant = True

   changeCommits = []
   for c in data[r]:
      irrelevant = True
      for k in allowedKeywords:
          if k.lower() in data[r][c]["keyword"].lower():
            #check if the keywords match with the ones we are looking for
            irrelevant = False
          
      if irrelevant:
        continue
        
      if not (".py" in data[r][c]["diff"]):
          #doesn't even contain a python file that was changed
          continue   

      if not "message" in data[r][c]:
          data[r][c]["message"] = ""
        
        #put the commit in a list to check if we get too many duplicates of the same commit (due to forks etc.)
      if not c in changedict:
          changedict[c] = 0
      if c in changedict:
          changedict[c] = changedict[c] + 1
      if changedict[c] > 5:
            #print(" we already have more than five. Skip.")
            continue
      else:
          changedict[c] = 1
        

        
      badparts = {}    
        
        #get all changes that are written down in the diff file
      changes = getChanges(data[r][c]["diff"])
        
      for change in changes:
          
          #make them into usable objects
          thischange = makechangeobj(change)

          if thischange is not None:
            if not "files" in data[r][c]:
              data[r][c]["files"] = {}
            f = thischange["filename"]
            
            if f is not None:
              
              #check the filename for hints that it is an implementation of an attack, a demonstration etc.
              suspicious = False
              for s in suspiciouswords:
                if s.lower() in f.lower():
                  #words should not appear in the filename
                  suspicious = True
                  
              if not suspicious:   
                if not f in data[r][c]["files"]:
                  data[r][c]["files"][f] = {}
                if not "changes" in data[r][c]["files"][f]:
                  data[r][c]["files"][f]["changes"] = []
                data[r][c]["files"][f]["changes"].append(thischange)
                changesfromdiff = True
                changeCommits.append(c)
            
      if changesfromdiff:
          #if any changes in this diff were useful...we get the sourcecode for those files using pydriller
          print("\n\n" + mode + "    mining "  + r + " " + str(progress) + "/" + str(len(data)))

          commitlist = []
          try:
            for commit in RepositoryMining(r).traverse_commits():
              commitlist.append(commit.hash)              

              #go through all commits in the repository mining and check if they match with one of the commits that are of interest
              if not commit.hash in changeCommits:
                continue
              
              for m in commit.modifications:                  
                  #run through all modifications in the single commit in the repository mining
                  if m.old_path != None and m.source_code_before != None:
                    if not ".py" in m.old_path:
                      continue
                    
                    #ignore files that are too large
                    if len(m.source_code_before) > 30000:
                      continue
                      
                    #print("\n  modification with old path: " + str(m.old_path))
                    for c in data[r]: 
                        if c == commit.hash:   
                          #run through commits we have for that repository until the match is found
                          print("  found commit " + c)
                          if not "files" in data[r][c]:
                            print("  no files :(") #rarely ever happens
                          data[r][c]["msg"] = commit.msg #get the commit message from the repository mining, check it for suspicious words
                          for badword in badwords:
                            if badword.lower() in commit.msg.lower():
                              suspicious = True
                          if suspicious:
                            print("  suspicious commit msg: \"" + commit.msg.replace("\n"," ")[:300] + "...\"")
                            continue
                          
                          #print("  The commit has " + str(len(data[r][c]["files"])) + " files.")
                          for f in data[r][c]["files"]:
                              
                              #find the file that was changed in the modification we are at
                              if m.old_path in f:

                                #grab the sourcecode and save it: before without comments, before with comments, and after with comments
                                if not ("source" in data[r][c]["files"][f] and (len(data[r][c]["files"][f]["source"])> 0)):
                                  sourcecode = "\n" + myutils.removeDoubleSeperatorsString(myutils.stripComments(m.source_code_before))
                                  data[r][c]["files"][f]["source"] = sourcecode

                                if not ("sourceWithComments" in data[r][c]["files"][f] and (len(data[r][c]["files"][f]["sourceWithComments"])> 0)):
                                  data[r][c]["files"][f]["sourceWithComments"] = m.source_code_before
                                
                                if not ("sourceWithComments" in data[r][c]["files"][f] and (len(data[r][c]["files"][f]["sourceWithComments"])> 0)):
                                  data[r][c]["files"][f]["sourcecodeafter"] = ""
                                  if m.source_code is not None:
                                    data[r][c]["files"][f]["sourcecodeafter"] = m.source_code   
                                
                                if not r in datanew:
                                  datanew[r] = {}
                                if not c in datanew[r]:
                                  datanew[r][c] = {}
                                  
                                #save it in the new dataset
                                datanew[r][c] = data[r][c]
                                print("     ->> added to the dataset.")
                                  
          except Exception as e:
              print("Exception occured.")
              print(e)
              time.sleep(2)
              continue



print("done.")
print(len(data))  

    #save dataset
with open('/content/drive/MyDrive/plain_' + mode, 'w') as outfile:
  json.dump(datanew, outfile)

OSError: ignored

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 4.3 MB/s 
     |████████████████████████████████| 59 kB 4.9 MB/s 
     |████████████████████████████████| 3.3 MB 38.8 MB/s 
     |████████████████████████████████| 895 kB 57.8 MB/s 
     |████████████████████████████████| 596 kB 42.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
from datetime import datetime
from pydriller import Repository
import json

import os.path
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

tokenizer = AutoTokenizer.from_pretrained('mrm8488/codebert-base-finetuned-detect-insecure-code')
model = AutoModelForSequenceClassification.from_pretrained('mrm8488/codebert-base-finetuned-detect-insecure-code')

with open('drive/MyDrive/plain_xss','r') as infile:
  data = json.load(infile)

currenttime = datetime.now()
formatcurrtime = currenttime.strftime("%H:%M")
print("Loading done. ",formatcurrtime)

progress=0
count = 0
blocks =[]

for i in data:
  project_url = data[i]["html_url"]
  for commit in Repository(project_url).traverse_commits:
    for m in commit.modified_files:
      inputs = tokenizer(str(m),return_tensors ="pt", truncation=True, padding='max_length')
      labels = torch.tensor([1]).unsqueeze(0)
      outputs = model(**inputs, labels = label)
      loss = outputs.loss
      logits = outputs.logits
      if np.argmax(logits.detach().numpy)==0:
        print(i["html_url"]," is vulnerable")


Loading done.  19:09


KeyError: ignored